In [1]:
import sys
import os
import pandas as pd
import numpy as np
import json
from sklearn.model_selection import train_test_split
import random as rn
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from numpy import array, savetxt, asarray, save
from keras_preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import LSTM, Input, TimeDistributed, Dense, Activation, RepeatVector, Embedding, Dropout
from keras.utils import np_utils
#from google.colab import drive
import tensorflow as tf
from keras.preprocessing.text import Tokenizer
from tensorflow import keras
from tensorflow.keras.callbacks import EarlyStopping
from keras.models import load_model
from keras.layers import GRU
#drive.mount('/content/drive')
import keras_tuner
from keras_tuner import HyperModel

2022-12-07 06:03:17.239612: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-07 06:03:18.010002: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/amazon/efa/lib:/opt/amazon/openmpi/lib:/usr/local/cuda/efa/lib:/usr/local/cuda/lib:/usr/local/cuda:/usr/local/cuda/lib64:/usr/local/cuda/extras/CUPTI/lib64:/usr/local/cuda/targets/x86_64-linux/lib:/usr/local/lib:/usr/lib:/opt/amazon/efa/lib:/opt/amazon/openmpi/lib:/usr/local/cuda/efa/lib:/usr/local/cuda/lib:/usr/local/cuda:/usr/local/cuda/lib64:/usr/local/cuda/extras/CUPTI/lib64:/usr/local/cud

In [2]:
DATA_PATH = "data/alzheimersdata.txt"
NEW_DATA = "data/TL_processed_data.txt"
TRAIN_DATA = "data/TL_train_data.txt"
TEST_DATA = "data/TL_test_data.txt"
VAL_DATA = "data/TL_val_data.txt"

Filter Dataset Into Smaller Segments

In [3]:
count=0
new = open(NEW_DATA, "w")
for line in open(DATA_PATH, "r"):
    count += 1
    if 30 < len(line) < 50 and count < 300000:
        if ('T' not in line) and ('V' not in line) and ('g' not in line) and ('L' not in line) and ('8' not in line):
            new.write(line)
file_new = np.array(list(open(NEW_DATA)))
print(file_new.shape)



(758,)


Find Maximum Sequence Length

In [4]:
file = open(NEW_DATA)
max_seq_len = int(len(max(file,key=len)))
print ("Max Sequence Length: ", max_seq_len)

Max Sequence Length:  49


In [5]:

#Define Functions
def read(fileName):
        fileObj = open(fileName, "r")
        words = fileObj.read().splitlines()
        fileObj.close()
        return words

def padFile(fileName):
  temp = read(fileName)
  preprocessed_pad_text = [['?'] + list(i) for i in temp]
  print("Sample 1: ", preprocessed_pad_text[0])
  print("Sample 2: ", preprocessed_pad_text[1])
  print("Sample 3: ", preprocessed_pad_text[2])
  print("Sample 4: ", preprocessed_pad_text[3])
  print("Sample 5: ", preprocessed_pad_text[4])
  padded_text = pad_sequences(preprocessed_pad_text, dtype=object, maxlen=max_seq_len+1, padding="post", value="!")
  # front_pad = [['?'] + i for i in padded_text]
  #print("Padded Text Arrays: ", padded_text)
  var = ["".join(i) for i in padded_text]
  print("Padded Strings: ", var[0:5])
  #print(var)
  # with open('/content/drive/MyDrive/CS230/padded_processed_data.txt', "w") as output:
  #   for x in var:
  #     output.write(y)
  # np.array(var)
  # np.savetxt('/content/drive/MyDrive/CS230/padded_processed_data.txt', var, fmt ='%s', newline='')
  return var
var = padFile(NEW_DATA)
var = np.array(var)



Sample 1:  ['?', '\ufeff', 'O', 'c', '1', 'c', 'c', 'c', '(', 'N', 'c', '2', 'n', 'c', 'n', 'c', '3', '[', 'n', 'H', ']', 'c', 'n', 'c', '2', '3', ')', 'c', 'c', '1', 'C', 'N', '(', 'C', 'C', 'C', 'l', ')', 'C', 'C', 'C', 'l']
Sample 2:  ['?', 'O', '=', 'N', 'c', '1', 'c', '(', '-', 'c', '2', 'c', '(', 'O', ')', '[', 'n', 'H', ']', 'c', '3', 'c', 'c', 'c', 'c', 'c', '2', '3', ')', '[', 'n', 'H', ']', 'c', '2', 'c', 'c', 'c', 'c', 'c', '1', '2']
Sample 3:  ['?', 'O', '=', 'N', 'c', '1', 'c', '(', '-', 'c', '2', 'c', '(', 'O', ')', '[', 'n', 'H', ']', 'c', '3', 'c', 'c', 'c', 'c', 'c', '2', '3', ')', '[', 'n', 'H', ']', 'c', '2', 'c', 'c', 'c', 'c', 'c', '1', '2']
Sample 4:  ['?', 'O', '=', 'N', 'c', '1', 'c', '(', '-', 'c', '2', 'c', '(', 'O', ')', '[', 'n', 'H', ']', 'c', '3', 'c', 'c', 'c', '(', 'I', ')', 'c', 'c', '2', '3', ')', '[', 'n', 'H', ']', 'c', '2', 'c', 'c', 'c', 'c', 'c', '1', '2']
Sample 5:  ['?', 'O', '=', 'N', 'c', '1', 'c', '(', '-', 'c', '2', 'c', '(', 'O', ')', '[', 

In [6]:
#Load & Save Datasets
#init random seed
seed = 1
np.random.seed(seed)
#split data into train/test
full_train, test = train_test_split(np.array(var), test_size=0.2, random_state=seed)
# full_train, test = train_test_split(np.array(var), test_size=0.25, random_state=seed)

In [7]:
#Split full train set into smaller train set and validation (dev) set
#np.savetxt('/content/drive/MyDrive/CS230/full_train_data.txt', full_train, fmt ='%s', newline='')
train, val = train_test_split(np.array(full_train), test_size=0.10, random_state=seed)
np.savetxt(TRAIN_DATA, train, fmt ='%s', newline='')
np.savetxt(TEST_DATA, test, fmt ='%s', newline='')
np.savetxt(VAL_DATA, val, fmt='%s', newline='')
print("Sample:", train[seed])
print("Train:", train.shape)
print("Validation:", val.shape)
print("Test:", test.shape)

Sample: ?Cn1cc(C2=C(C(=O)CBr)C(=O)NC2=O)c2ccccc21!!!!!!!!!
Train: (545,)
Validation: (61,)
Test: (152,)


Train Data Processing

In [8]:
#Load Data (optional: only if previous cells are not run and data is saved already)
# train = pd.read_fwf('/content/drive/MyDrive/CS230/train_data.txt')
#Concatenate Data
def concatenate(data):
    #res = ''
    print(len(data))
    #for count, word in enumerate(data):
        #if count %1000 == 0:
            #print(count)
            #pass
        # print(word)
        #res += word
    res = ''.join(data)
    return res
train = concatenate(train)
print(train[0:100])
#Tokenize Data
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=None, char_level=True, lower=False)
tokenizer.fit_on_texts(train)
new_train = tokenizer.texts_to_sequences(train)
print(new_train[0:100])
print(train[0:100])
#Print Data Breakdown
n_chars = len(train)
n_vocab = len(list(set(train)))
print("# of Unique Characters:", n_chars)
print("# of Total Characters:", n_vocab)
n_chars = len(new_train)
n_vocab = len(set(train))
print("# of Unique Characters:", n_chars)
print("# of Total Characters:", n_vocab)

#N-Grams Sequence
seqLen = 15
stepSize = 1
input_chars = []
next_char = []
for i in range(0, len(new_train) - seqLen, stepSize):
  input_chars.append(new_train[i : i + seqLen])
  next_char.append(new_train[i + seqLen])
for i in range(5):
  print("Input Sequence:", input_chars[i])
  print("Next Character Prediction:", next_char[i])
  
#Assemble Train Datasets
x_train = np.array(input_chars)
x_train.flatten()
y_train = np.array(next_char)
y_train_2 = np_utils.to_categorical(y_train)
print(f"x_train shape {x_train.shape}")
print(f"y_train shape {y_train.shape}")
print(f"x_train {x_train[0:5]}")
print(f"y_train {y_train[0:5]}")

#Tokenize Data
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=None, char_level=True, lower=False)
tokenizer.fit_on_texts(val)
new_val = tokenizer.texts_to_sequences(val)
print(new_val[0:100])
print(val[0:100])

#Print Data Breakdown
n_chars = len(val)
n_vocab = len(list(set(val)))
print("# of Unique Characters:", n_chars)
print("# of Total Characters:", n_vocab)
n_chars = len(new_val)
n_vocab = len(set(val))
print("# of Unique Characters:", n_chars)
print("# of Total Characters:", n_vocab)
print("Difference", set(train).difference(set(val)))

#N-Grams Sequence
seqLen = 15
stepSize = 1
input_chars = []
next_char = []
for i in range(0, len(new_val) - seqLen, stepSize):
  input_chars.append(new_val[i : i + seqLen])
  next_char.append(new_val[i + seqLen])
#print(set(next_char.items))
for i in range(5):
  print("Input Sequence:", input_chars[i])
  print("Next Character Prediction:", next_char[i])

#Assemble Validation Datasets
x_val = np.array(input_chars)
x_val.flatten()
y_val = np.array(next_char)
y_val_2 = np_utils.to_categorical(y_val)
print(y_val_2)
print(y_train_2)
#score = model.evaluate(x_val, y_val_2)
#print(score[1])
#print(result)

545
?COc1cccc(C2=C(c3cn(C)c4ccc(Cl)cc34)C(=O)NC2=O)c1!?Cn1cc(C2=C(C(=O)CBr)C(=O)NC2=O)c2ccccc21!!!!!!!!!
[[13], [3], [9], [1], [6], [1], [1], [1], [1], [4], [3], [7], [11], [3], [4], [1], [10], [1], [8], [4], [3], [5], [1], [17], [1], [1], [1], [4], [3], [22], [5], [1], [1], [10], [17], [5], [3], [4], [11], [9], [5], [12], [3], [7], [11], [9], [5], [1], [6], [2], [13], [3], [8], [6], [1], [1], [4], [3], [7], [11], [3], [4], [3], [4], [11], [9], [5], [3], [25], [27], [5], [3], [4], [11], [9], [5], [12], [3], [7], [11], [9], [5], [1], [7], [1], [1], [1], [1], [1], [7], [6], [2], [2], [2], [2], [2], [2], [2], [2], [2]]
?COc1cccc(C2=C(c3cn(C)c4ccc(Cl)cc34)C(=O)NC2=O)c1!?Cn1cc(C2=C(C(=O)CBr)C(=O)NC2=O)c2ccccc21!!!!!!!!!
# of Unique Characters: 27250
# of Total Characters: 34
# of Unique Characters: 27250
# of Total Characters: 34
Input Sequence: [[13], [3], [9], [1], [6], [1], [1], [1], [1], [4], [3], [7], [11], [3], [4]]
Next Character Prediction: [1]
Input Sequence: [[3], [9], [1], [6], [

In [ ]:
transferLearned_BaselineLSTM = keras.models.load_model('baseline_lstm.h5')
transferLearned_BaselineLSTM.layers[0].trainable = False
transferLearned_BaselineLSTM.layers[1].trainable = False
transferLearned_BaselineLSTM.fit(x_train, y_train_2, epochs = 40, batch_size = 128, validation_data=(x_val, y_val_2), verbose=1, callbacks = [early])
transferLearned_Hybrid = keras.models.load_model('hybrid.h5')
transferLearned_Hybrid.layers[0].trainable = False
transferLearned_Hybrid.layers[1].trainable = False
transferLearned_Hybrid.fit(x_train, y_train_2, epochs = 40, batch_size = 128, validation_data=(x_val, y_val_2), verbose=1, callbacks = [early])